In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels as stats

In [37]:
df = pd.read_csv('../data/ufo_data/ufo-sightings-transformed.csv')
df.head()

,Unnamed: 0,Date_time,date_documented,Year,Month,Hour,Season,Country_Code,Country,Region,Locale,latitude,longitude,UFO_shape,length_of_encounter_seconds,Encounter_Duration,Description
0,0,1949-10-10 20:30:00,4/27/2004,1949,10,20,Autumn,USA,United States,Texas,San Marcos,29.883056,-97.941111,Cylinder,2700.0,45 minutes,This event took place in early fall around 194...
1,1,1949-10-10 21:00:00,12/16/2005,1949,10,21,Autumn,USA,United States,Texas,Bexar County,29.384210,-98.581082,Light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...
2,2,1955-10-10 17:00:00,1/21/2008,1955,10,17,Autumn,GBR,United Kingdom,England,Chester,53.200000,-2.916667,Circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...
3,3,1956-10-10 21:00:00,1/17/2004,1956,10,21,Autumn,USA,United States,Texas,Edna,28.978333,-96.645833,Circle,20.0,1/2 hour,My older brother and twin sister were leaving ...
4,4,1960-10-10 20:00:00,1/22/2004,1960,10,20,Autumn,USA,United States,Hawaii,Kaneohe,21.418056,-157.803611,Light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80328 entries, 0 to 80327
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   80328 non-null  int64  
 1   Date_time                    80328 non-null  object 
 2   date_documented              80328 non-null  object 
 3   Year                         80328 non-null  int64  
 4   Month                        80328 non-null  int64  
 5   Hour                         80328 non-null  int64  
 6   Season                       80328 non-null  object 
 7   Country_Code                 80069 non-null  object 
 8   Country                      80069 non-null  object 
 9   Region                       79762 non-null  object 
 10  Locale                       79871 non-null  object 
 11  latitude                     80328 non-null  float64
 12  longitude                    80328 non-null  float64
 13  UFO_shape       

In [39]:
df['UFO_shape'].value_counts()

UFO_shape
Light        16565
Triangle      7865
Circle        7607
Fireball      6208
Other         5649
Unknown       5584
Sphere        5387
Disk          5213
Oval          3733
Formation     2457
Cigar         2057
Changing      1962
Flash         1328
Rectangle     1296
Cylinder      1283
Diamond       1178
Chevron        952
Egg            759
Teardrop       750
Cone           316
Cross          233
Delta            7
Round            2
Crescent         2
Pyramid          1
Flare            1
Hexagon          1
Dome             1
Changed          1
Name: count, dtype: int64

In [43]:
df_cleaned = df
df_cleaned['UFO_shape'] = df_cleaned['UFO_shape'].fillna('Unknown')



In [52]:

def shape_binner(shape_list):
    binned_shapes = []
    for shape in shape_list:
        if shape in ['Light', 'Fireball', 'Flash', 'Flare']:
            binned_shapes.append('Light-Only')
        elif shape in ['Triangle', 'Diamond', 'Teardrop', 'Cone', 'Delta','Pyramid']:
            binned_shapes.append('Tapered')
        elif shape in ['Circle', 'Sphere','Disk','Oval','Egg','Dome','Round']:
            binned_shapes.append('Rounded')
        elif shape in ['Formation', 'Hexagon']:
            binned_shapes.append('Other')
        elif shape in ['Cigar','Rectangle','Cylinder']:
            binned_shapes.append('Linear-Symmetric')
        elif shape in ['Chevron','Crescent','Cross']:
            binned_shapes.append('Winged')
        elif shape == 'Changed':
            binned_shapes.append('Changing')
        else:
            binned_shapes.append(shape)
    return binned_shapes

In [53]:
df_cleaned['UFO_shape_binned'] = shape_binner(df['UFO_shape'])

In [54]:
df.groupby('UFO_shape_binned').mean(numeric_only=True)

,Unnamed: 0,Year,Month,Hour,latitude,longitude,length_of_encounter_seconds
UFO_shape_binned,,,,,,,
Changing,38939.025981,2005.154356,6.726439,14.872644,37.661640,-87.909169,2080.287978
Light-Only,40468.521451,2005.723467,6.931375,16.241308,38.197707,-87.709329,10381.962252
Linear-Symmetric,41484.553494,2001.739862,6.793788,14.804789,37.931360,-85.808835,1989.994187
Other,39157.707907,2003.732700,6.881954,15.400765,38.089282,-86.528732,14758.122004
Rounded,40788.216457,2002.431372,6.772443,15.206722,37.931986,-85.239665,7807.184082
Tapered,39404.842641,2004.043491,6.828408,15.532075,38.295914,-86.368809,4116.996563
Unknown,39579.326324,2002.987490,6.701357,14.812883,38.565359,-88.927188,16186.370257
Winged,35920.932603,2003.668913,6.998315,16.267060,37.859853,-90.383565,547.967102


In [31]:
regions = df['Region'].value_counts()
regions[700:743]

Region
Norte de Santander                     1
Kharkiv Oblast                         1
Caloocan                               1
Vorarlberg                             1
Petén                                  1
Chaharmahal and Bakhtiyari Province    1
Omaheke                                1
Homs Governorate                       1
Hainan Province                        1
County Tipperary                       1
Lusaka Province                        1
Al Asimah                              1
Silesian Voivodeship                   1
Ras al-Khaimah                         1
Ialomița                               1
County Leitrim                         1
Querétaro                              1
Seo-gu                                 1
Plaines Wilhems                        1
Ajman Emirate                          1
Macau                                  1
Colón                                  1
Barahona                               1
Siem Reap                              1
Asturias 

In [35]:
df['UFO_shape'].value_counts()

UFO_shape
Light        16565
Triangle      7865
Circle        7607
Fireball      6208
Other         5649
Unknown       5584
Sphere        5387
Disk          5213
Oval          3733
Formation     2457
Cigar         2057
Changing      1962
Flash         1328
Rectangle     1296
Cylinder      1283
Diamond       1178
Chevron        952
Egg            759
Teardrop       750
Cone           316
Cross          233
Delta            7
Round            2
Crescent         2
Pyramid          1
Flare            1
Hexagon          1
Dome             1
Changed          1
Name: count, dtype: int64